# <font style="color:blue">4. Loss Function</font>

Let's see how our loss function should look like.

Different from classification and semantic segmentation task, we now have two branches with two independent outputs.

At each iteration, we want to know 2 things - how close are the predicted bounding boxes to the target bounding boxes, and whether their labels are predicted correctly.

That's why our loss function is represented as a sum of two losses: localization and classification loss.

In [1]:
from IPython.display import Code
import inspect

from detection_loss import DetectionLoss

In [2]:
Code(data=inspect.getsource(DetectionLoss)[:797])

class DetectionLoss(nn.Module):
    def __init__(self, num_classes=2):
        super().__init__()
        self.num_classes = num_classes

    def forward(self, loc_preds, loc_targets, cls_preds, cls_targets):
        '''Compute loss between (loc_preds, loc_targets) and (cls_preds, cls_targets).

        Args:
          loc_preds: (tensor) predicted locations, sized [batch_size, #anchors, 4].
          loc_targets: (tensor) encoded target locations, sized [batch_size, #anchors, 4].
          cls_preds: (tensor) predicted class confidences, sized [batch_size, #anchors, #classes].
          cls_targets: (tensor) encoded target labels, sized [batch_size, #anchors].

        loss:
          (tensor) loss = SmoothL1Loss(loc_preds, loc_targets) + OHEMLoss(cls_preds, cls_targets).
        '''

## <font style="color:green">4.1. Localization Loss</font>

For localization loss, we choose Smooth L1-loss, following [Faster R-CNN paper](https://arxiv.org/pdf/1506.01497.pdf).

It was invented as a solution for bounding box regression problem that L2 loss suffers from, as it is sensitive to outliers.

Smooth L1-loss can be interpreted as a combination of L1-loss and L2-loss.

It behaves as L1-loss when the absolute value of the argument is high, and like L2-loss when the absolute value of the argument is close to zero.

<img src='https://www.learnopencv.com/wp-content/uploads/2020/03/c3-w8-l1l2smoothl1.png' align='middle'>

In [3]:
Code(data=inspect.getsource(DetectionLoss)[962:1420])

pos = cls_targets > 0  # [N,#anchors]
        num_pos = pos.long().sum(1, keepdim=True)

        mask = pos.unsqueeze(2).expand_as(loc_preds)  # [N,#anchors,4]
        masked_loc_preds = loc_preds[mask].view(-1, 4)  # [#pos,4]
        masked_loc_targets = loc_targets[mask].view(-1, 4)  # [#pos,4]
        loc_loss = F.smooth_l1_loss(masked_loc_preds, masked_loc_targets, reduction='none')
        loc_loss = loc_loss.sum() / num_pos.sum().float()

## <font style="color:green">4.2. Classification Loss</font>

As classification loss, we use Cross Entropy Loss as the most popular loss for classification task.

It should be noted, that class imbalance is a very problematic issue for single-stage detectors.
This is because most locations in an image are negatives, that can be easily classified by the detector as background.

We want out network to train on hard examples with positives, which constitute only a small part of all of the locations.

There are different methods on how to deal with that issue. We choose to use Online Hard Example Mining (OHEM) strategy.
It finds hard examples in the batch with the greatest loss values and back-propagates the loss computed over the selected instances.
The amount of hard examples correlates with the number of positive examples and is often chosen as `3:1`.

In [4]:
Code(data=inspect.getsource(DetectionLoss)[1597:3835])

# Compute max conf across batch for hard negative mining
        batch_size, _ = cls_targets.size()
        batch_conf = cls_preds.view(-1, self.num_classes)
        cls_loss = F.cross_entropy(batch_conf, cls_targets.view(-1), ignore_index=-1, reduction='none')
        cls_loss = cls_loss.view(batch_size, -1)

        # Hard Negative Mining
        # filter out pos boxes (pos = cls_targets > 0) for now.
        pos_cls_loss = cls_loss[pos]
        
        # In OHEM, we have to select only those background labels (0) that have been failed with 
        # a very high margin (lets we will choose three times (negpos_ratio = 3) of the object labels (>=1)). 
        
        # To paly around background labels, let's make zero loss to object labels.
        cls_loss[pos] = 0 
        
        # Let's find indices of decreasing order of loss (which ground truth is background). 
        _, loss_idx = cls_loss.sort(1, descending=True)
        
        # If we sort (in increasing order) the above indices, indices correspond to the sorting will 
        # give a ranking (along dimension 1) of the original loss matrix. 
        _, idx_rank = loss_idx.sort(1)
        
        # Let's understand by example. As all operations are along axis 1, taking 1-d example will be sufficient.

        # cls_loss = [5, 2, 9, 6,  8]

        # _, loss_idx = cls_loss.sort(descending=True)
        # loss_idx = [2, 4, 3, 0, 1]

        # _, idx_rank = loss_idx.sort()
        # idx_rank = [3, 4, 0, 2, 1]
        
        # Have a look, idx_rank has the ranking of cls_loss.

        negpos_ratio = 3
        
        # We have decided we will take the negative class count three times of the positive class. 

        # If we do it blindly, in the case of not a positive class in the image, we will end up missing 
        # all the negative class also. So let's clamp minimum to 1. 
        # Although maximum clamping is not required here,  let fix to maximum index. 

        num_neg = torch.clamp(negpos_ratio * num_pos, min=1, max=pos.size(1) - 1)

        neg = idx_rank < num_neg.expand_as(idx_rank)
        neg_cls_loss = cls_loss[neg]

        cls_loss = (pos_cls_loss.sum() + neg_cls_loss.sum()) / num_pos.sum().float()

Our final loss will be a weighted sum of localization and classification loss:

In [5]:
Code(data=inspect.getsource(DetectionLoss)[3835:])

# The magnitude of cross-entropy loss is much more than L2, L1, and smooth L1. 
        # So it is better to take a weighted loss. Here we have chosen twenty times of 
        # lower magnitude loss and one time of higher magnitude loss.

        return 20 * loc_loss, cls_loss